# US-Ignite San Diego ETL Notebook

In [43]:
#import libraries
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer
from cartoframes.data import Dataset
from cartoframes.viz.helpers import color_continuous_layer
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# import CityIq and libraries for timing
from cityiq import CityIq
import time
import json

#load .env with credentials
from dotenv import load_dotenv
load_dotenv()

import os
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("API_KEY")
USERNAME = os.getenv("USERNAME")

set_default_credentials(
    base_url=BASE_URL,
    api_key=API_KEY
)

credentials = Credentials(USERNAME, API_KEY)


## Extract: Bring Data into Notebook

In [44]:
# set time frame for use when querying for events (epoch time in milliseconds)
endTime = int(time.time())*1000 # time when demo.py is run
startTime = endTime-3600000 # startTime is 1 hour before endTime

# get CityIq token
myCIQ = CityIq("City")
myCIQ.fetchToken()

# getting assets - assets with PEDEVT events
myCIQ.fetchMetadata("assets","pedestrian","eventTypes:PEDEVT")
san_diego_pedestrian_sensor_metadata = myCIQ.getAssets()

## Transform
Add here some code to do data transformation (remove outliers, null values, etc.). It's probably best to do this type of work on the geopandas dataframe 

In [45]:
san_diego_pedestrian_sensor_metadata_df = pd.DataFrame(san_diego_pedestrian_sensor_metadata)
san_diego_pedestrian_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_pedestrian_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 

san_diego_pedestrian_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_pedestrian_sensor_metadata_df["longitude"]= latlng[1].astype(float)
#print(san_diego_pedestrian_sensor_metadata_df)

san_diego_pedestrian_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_pedestrian_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_pedestrian_sensor_metadata_df.longitude, san_diego_pedestrian_sensor_metadata_df.latitude))
print(san_diego_pedestrian_sensor_metadata_gdf)

   assetType                              assetUid  \
0     CAMERA  00423a29-2ad6-4776-9bb6-c9be6e31a964   
1     CAMERA  03a7eaf3-c758-4526-b8a4-1db39bb6697c   
2     CAMERA  05330a7b-5b72-4135-9680-b650ede1cb32   
3     CAMERA  06fc5f10-5081-41e7-bf25-e71728ae18fb   
4     CAMERA  0b977f7b-92d4-4081-a09f-5274a4489758   
5     CAMERA  0de8fc8b-8078-4005-9303-a6433be727d1   
6     CAMERA  139e7832-20f2-46c0-823d-6fb4cde7f11c   
7     CAMERA  18ba13cd-5070-419d-a713-06fba3ed047c   
8     CAMERA  18fa1950-1e69-4097-bcad-634e73bb1ec5   
9     CAMERA  1a059929-fdc8-4c8a-81e3-0e72c75e40aa   
10    CAMERA  28d2b864-c954-4686-9f00-76f69a5a5705   
11    CAMERA  2ffd6259-98f0-491a-8257-10374ac2c041   
12    CAMERA  321a03e9-0146-4d29-95b0-6fc9201f6308   
13    CAMERA  32659876-9f3f-4b5a-bc93-89e1926f3f62   
14    CAMERA  327867d7-02af-41f1-9ccf-4874eb002a54   
15    CAMERA  33ac5ea9-38e0-41f1-b7d1-a8af745a0f4a   
16    CAMERA  34cd9af8-b0ae-4389-bd9d-4e3a1368612b   
17    CAMERA  3cd5c384-4c6e-

## Load: Upload to CARTO

In [46]:
san_diego_pedestrian_sensor_metadata_d = Dataset(san_diego_pedestrian_sensor_metadata_gdf)
san_diego_pedestrian_sensor_metadata_d.upload(table_name='san_diego_pedestrian_sensor_metadata', if_exists='replace', credentials=credentials)

The following columns were changed in the CARTO copy of this dataframe:
assetType -> assettype
assetUid -> assetuid
eventTypes -> eventtypes
mediaType -> mediatype
parentAssetUid -> parentassetuid


/anaconda3/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 56021a86-caf1-484e-a4f4-cc5287ea417e
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


## ETL for loading Events

In [47]:
# empty list to collect events
san_diego_pedestrian_sensor_events_list = []
# loop through assets to fetch events for each asset
for index, row in san_diego_pedestrian_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "PEDEVT", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
    for a in assetEvents:
        a["latitude"] = row.latitude
        a["longitude"] = row.longitude
        a["directionUnit"] = a["properties"]["directionUnit"]
        a["speedUnit"] = a["properties"]["speedUnit"]
        a["eventUid"] = a["properties"]["eventUid"]
        a["counter_direction_speed"] = a["measures"]["counter_direction_speed"]
        a["counter_direction_pedestrianCount"] = a["measures"]["counter_direction_pedestrianCount"]
        a["pedestrianCount"] = a["measures"]["pedestrianCount"]
        a["counter_direction"] = a["measures"]["counter_direction"]
        a["speed"] = a["measures"]["speed"]
        a["direction"] = a["measures"]["direction"]
        san_diego_pedestrian_sensor_events_list.append(a)

#     assetEvents_df = pd.DataFrame(assetEvents)
#     assetEvents_df["latitude"] = row.latitude
#     assetEvents_df["longitude"] = row.longitude
#     san_diego_pedestrian_sensor_events_list.append(assetEvents)
    
# print(san_diego_pedestrian_sensor_events_list)
san_diego_pedestrian_sensor_events_df = pd.DataFrame(san_diego_pedestrian_sensor_events_list)
san_diego_pedestrian_sensor_events_df.dropna(inplace = True)
# print(san_diego_pedestrian_sensor_events_df)
    
    
san_diego_pedestrian_sensor_events_gdf = gpd.GeoDataFrame(san_diego_pedestrian_sensor_events_df, geometry=gpd.points_from_xy(san_diego_pedestrian_sensor_events_df.longitude, san_diego_pedestrian_sensor_events_df.latitude))
san_diego_pedestrian_sensor_events_d = Dataset(san_diego_pedestrian_sensor_events_gdf)
san_diego_pedestrian_sensor_events_d.upload(table_name='san_diego_pedestrian_sensor_events', if_exists='append', credentials=credentials)

    

The following columns were changed in the CARTO copy of this dataframe:
assetUid -> assetuid
counter_direction_pedestrianCount -> counter_direction_pedestriancount
directionUnit -> directionunit
eventType -> eventtype
eventUid -> eventuid
locationUid -> locationuid
pedestrianCount -> pedestriancount
speedUnit -> speedunit


## Make a quick map using one of CARTO's helper functions

In [48]:
Map(Layer(san_diego_pedestrian_sensor_metadata_d))